In [ ]:
import pandas as pd
import os
import wget
import PyPDF2
from PyPDF2 import PdfReader
import urllib.parse

In [ ]:
df = pd.read_csv("Studentenbewegung_free_pdf.csv", encoding="utf-8")
df

### Linkliste erstellen

In [ ]:
links = df["links"]
print(links)

In [ ]:
print(links[0])

In [ ]:
linklist = []

for element in links:
    related_links = element.split("; ")
    for link in related_links: 
        if link.startswith("https://d-nb.info/"):
            #print(link)
            linklist.append(link)

In [ ]:
print(len(linklist))

#### Alternative:

```
linklist = []

for idn in df.idn: 
    link = "https://d-nb.info/" + idn + "/34"
    linklist.append(link) 
```

## Download der PDFs

Gekürzte Liste erstellen (damit es schneller geht):

In [ ]:
testlist = linklist[60:70]
print(testlist)

Funktion zum Herunterladen definieren, die zunächst ein neues Verzeichnis für die gesammelten Dateien erstellt und diese dann herunterlädt und unter Nutzung der IDN als Dateiname abspeichert:

In [ ]:
import requests

def download_text(testlist, save_directory):
    # Erstellen des später definierten Speicherverzeichnis, falls es dieses noch nicht gibt
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    for link in testlist: 
        # IDN ermitteln:
        idn = link.split('/')[-2]
        # Original-Dateiname und -Endung aus Content-Disposition header ermitteln und als Basis nutzen:
        response = requests.head(link)
        content_disposition = response.headers.get('Content-Disposition')
        orig_filename = content_disposition.split('filename=')[-1].strip('";')
        orig_extension = orig_filename.split(".")[-1]
        #print(orig_extension)

        # Dateiname aus IDN und originalem Dateiname zusammensetzen: 
        file_name = f"{idn}.{orig_extension}"
        file_path = os.path.join(save_directory, file_name)
        print(f" Starte Download: {file_name}")
        try: 
            #Datei herunterladen:
            wget.download(link, out=file_path)
        except Exception as e:
            print(f"Fehler beim Herunterladen von {link}: {e}")

In [ ]:
save_directory = "downloads"

download_text(testlist, save_directory)